# Installation

## Env setup

In [1]:
!conda env create -f environment_dev.yml
!conda activate AutoPrompt

WARNING conda.models.version:get_matcher(556): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.8.0.*, but conda is ignoring the .* and treating it as 1.8.0
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0



datasets-3.1.0       | 325 KB    |                                       |   0% 
pyarrow-core-18.0.0  | 3.7 MB    |                                       |   0% 

snappy-1.2.1         | 35 KB     |                                       |   0% 


aws-c-sdkutils-0.2.1 | 49 KB     |                                       |   0% 



orc-2.0.3            | 428 KB    |                                       |   0% 




libedit-3.1.20191231 | 94 

In [2]:
!pip install -r requirements.txt

  Obtaining dependency information for langchain==0.1.9 from https://files.pythonhosted.org/packages/45/05/2d5c38c7fa1a091a61e018070b6878410d256be577e5154843d339e59b2c/langchain-0.1.9-py3-none-any.whl.metadata
  Using cached langchain-0.1.9-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for openai==1.1.0 from https://files.pythonhosted.org/packages/77/b4/8de305dd47c13e3ac2e8eef6dc1ebc6f0884d4ac67af9f86d612c949ab63/openai-1.1.0-py3-none-any.whl.metadata
  Using cached openai-1.1.0-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for tiktoken==0.5.1 from https://files.pythonhosted.org/packages/fb/2a/3d02ef030f387c373acbeca6d5a2307405a1da735285ec12a9ed0b6302ea/tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Obtaining dependency information for transformers==4.35.2 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c

## Human Anotator setup

In [3]:
!pip install argilla

In [5]:
import argilla as rg

## Weights & Bias setup

In [ ]:
## copy this into the terminal
# !pip install wandb
# !wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [3]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dansashadan. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc,▁▅▇▇▇███
loss,█▅▅▄▁▂▁▂
acc,0.74977
loss,0.24823


# First Example- Filter Movie Reviews

In [5]:
from optimization_pipeline import OptimizationPipeline
from utils.config import load_yaml, override_config
import argparse


In [10]:

# General Training Parameters
parser = argparse.ArgumentParser()

parser.add_argument('--basic_config_path', default='config/config_default.yml', type=str, help='Configuration file path')
parser.add_argument('--batch_config_path', default='',
                    type=str, help='Batch classification configuration file path')
parser.add_argument('--prompt',
                    default='',
                    required=False, type=str, help='Prompt to use as initial.')
parser.add_argument('--task_description',
                    default='',
                    required=False, type=str, help='Describing the task')
parser.add_argument('--load_path', default='', required=False, type=str, help='In case of loading from checkpoint')
parser.add_argument('--output_dump', default='dump', required=False, type=str, help='Output to save checkpoints')
parser.add_argument('--num_steps', default=40, type=int, help='Number of iterations')

opt = parser.parse_args()

# # if opt.batch_config_path == '':
# #     # load the basic configuration using load_yaml
# #     config_params = load_yaml(opt.basic_config_path)
# # else:
# #     # override the basic configuration with the batch configuration
# #     config_params = override_config(opt.batch_config_path, config_file=opt.basic_config_path)

# # if opt.task_description == '':
# #     task_description = input("Describe the task: ")
# # else:
# #     task_description = opt.task_description

# # if opt.prompt == '':
# #     initial_prompt = input("Initial prompt: ")
# # else:
# #     initial_prompt = opt.prompt

# # # # Initializing the pipeline
# # # pipeline = OptimizationPipeline(config_params, task_description, initial_prompt, output_path=opt.output_dump)
# # # if (opt.load_path != ''):
# # #     pipeline.load_state(opt.load_path)
# # # best_prompt = pipeline.run_pipeline(opt.num_steps)
# # # print('\033[92m' + 'Calibrated prompt score:', str(best_prompt['score']) + '\033[0m')
# # # print('\033[92m' + 'Calibrated prompt:', best_prompt['prompt'] + '\033[0m')



usage: ipykernel_launcher.py [-h] [--basic_config_path BASIC_CONFIG_PATH]
                             [--batch_config_path BATCH_CONFIG_PATH]
                             [--prompt PROMPT]
                             [--task_description TASK_DESCRIPTION]
                             [--load_path LOAD_PATH]
                             [--output_dump OUTPUT_DUMP]
                             [--num_steps NUM_STEPS]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/ogc2020/Library/Jupyter/runtime/kernel-v3de9345fe9bb4f32804ed5b237394895a4cb2e5a2.json


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/ogc2020/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3505 in   │
│ run_code                                                                                         │
│                                                                                                  │
│   3502 │   │   │   │   if async_:                                                                │
│   3503 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3504 │   │   │   │   else:                                                                     │
│ ❱ 3505 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3506 │   │   │   finally:                                                                      │
│   3507 │   │   │   │   # Reset our crash handler in place                                        │
│   3508 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 parser.add_argument('--output_dump', default='dump', required=False, type=str, help='Out    │
│   15 parser.add_argument('--num_steps', default=40, type=int, help='Number of iterations')       │
│   16                                                                                             │
│ ❱ 17 opt = parser.parse_args()                                                                   │
│   18                                                                                             │
│   19 # # if opt.batch_config_path == '':                                                         │
│   20 # #     # load the basic configuration using load_yaml                                      │
│                                                                                                  │
│ /Users/ogc2020/anaconda3/lib/python3.11/argparse.py:1872 in parse_args                           │
│                                                                                                  │
│   1869 │   │   args, argv = self.parse_known_args(args, namespace)                               │
│   1870 │   │   if argv:                                                                          │
│   1871 │   │   │   msg = _('unrecognized arguments: %s')                                         │
│ ❱ 1872 │   │   │   self.error(msg % ' '.join(argv))                                              │
│   1873 │   │   return args                                                                       │
│   1874 │                                                                                         │
│   1875 │   def parse_known_args(self, args=None, namespace=None):                                │
│                                                                                                  │
│ /Users/ogc2020/anaconda3/lib/python3.11/argparse.py:2628 in error                                │
│                                                                                                  │
│   2625 │   │   """                                                                               │
│   2626 │   │   self.print_usage(_sys.stderr)                                                     │
│   2627 │   │   args = {'prog': self.prog, 'message': message}                                    │
│ ❱ 2628 │   │   self.exit(2, _('%(prog)s: error: %(message)s\n') % args)                          │
│   2629                                                                                           │
│                                                            